## Imports

In [34]:
import numpy as np
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from mss import mss
import cv2
import tensorflow as tf
sct = mss()
MODEL_FILE = 'ssd_mobilenet_v1_coco_2018_01_28.tar.gz'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'frozen_inference_graph.pb'
PATH_TO_LABELS= 'mscoco_label_map.pbtxt'
detection_graph = tf.Graph()
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Middle Location

In [35]:
ACTIVATION_RANGE = 400
origbox = (int(1920/2 - ACTIVATION_RANGE/2),
               int(1080/2 - ACTIVATION_RANGE/2),
               int(1920/2 + ACTIVATION_RANGE/2),
               int(1080/2 + ACTIVATION_RANGE/2))


## Main Loop

In [36]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    while True:
        screen = cv2.resize(grab_screen(region=(0,0,WIDTH,HEIGHT)), (int(WIDTH/RESIZE_FACTOR),int(HEIGHT/RESIZE_FACTOR)))
        image_np = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections])
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=3)
        cv2.imshow('Press q to quit bot',image_np)

KeyError: "The name 'detection_boxes:0' refers to a Tensor which does not exist. The operation, 'detection_boxes', does not exist in the graph."